In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

In [ ]:
import matplotlib.pyplot as plt
import glob
import cv2
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
train_labels = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
print('Shape of train dataset :',train_labels.shape)
train_labels.head(5)

In [ ]:
train_labels['target'].value_counts()

# We have class imbalance

In [ ]:
class ET_Dataset(Dataset):
    
    def __init__(self, img_paths, targets, resize=None, augmentations=None):
        self.img_paths = img_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, item):
        img = np.load(self.img_paths[item]).astype('float')
        target = self.targets[item]
        
        if self.resize is not None:
            img = np.transpose(img, (1, 2, 0))
            img = cv2.resize(img, dsize=self.resize, interpolation=cv2.INTER_CUBIC)
        
        if self.augmentations is not None:
            aug = self.augmentations(image = img)
            img = augmented["img"]
        
        img = np.transpose(img, (2, 0, 1)).astype(np.float32)
        
        return {"image":torch.tensor(img, dtype=torch.float),
               "target":torch.tensor(target, dtype=torch.long),}

In [ ]:
train_labels['img_path'] = train_labels['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
train_labels.head()

In [ ]:
def train_model(data_loader, model, optimizer, criterion, device):
    loss_arr = []
    # Setting model to train mode
    model.train()
    
    for data in tqdm(data_loader, leave=True, desc='Training'):
        inputs = data["image"]
        target = data["target"]
        
        inputs = inputs.to(device, dtype=torch.float)
        target = target.to(device, dtype=torch.float)
        
        # Forward Pass
        output = model(inputs)
        loss = criterion(output, target.view(-1, 1))
        
        # Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_arr.append(loss.item())
        
        del inputs, target, output
    return round(sum(loss_arr)/len(loss_arr), 3)

In [ ]:
def evaluate_model(data_loader, model, device):
    
    # Set the model to evaluation mode
    model.eval()
    
    _actual = []
    _preds = []
    
    with torch.no_grad():
        for data in tqdm(data_loader, leave=True, desc='Evaluating'):
            inputs = data["image"]
            target = data["target"]
            
            inputs = inputs.to(device, dtype=torch.float)
            target = target.to(device, dtype=torch.float)
            
            pred = model(inputs)
            
            target = target.detach().cpu().numpy().tolist()
            pred = pred.detach().cpu().numpy().tolist()
            
            _actual.extend(target)
            _preds.extend(pred)
    
    return metrics.roc_auc_score(_actual, _preds)

In [ ]:
def select_model(model_name, pretrained=True):
    
    if model_name.lower()=="resnext50":
        if pretrained:
            return models.resnext50_32x4d(pretrained=True)
        else:
            return models.resnext50_32x4d(pretrained=False)
    elif model_name.lower()=="resnet18":
        if pretrained:
            return models.resnet18(pretrained=True)
        else:
            return model.resent18(pretrained=False)
    
    elif model_name.lower()=="resnet34":
        if pretrained:
            return models.resnet34(pretrained=True)
        else:
            return model.resent34(pretrained=False)
    
    elif model_name.lower()=="resnet50":
        if pretrained:
            return models.resnet50(pretrained=True)
        else:
            return model.resent50(pretrained=False)
    

In [ ]:
def get_configured_model_resnet(model_name, pretrained=True):
    model = select_model(model_name, pretrained)
    model.fc = nn.Sequential(
        nn.BatchNorm1d(512),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=512, out_features=1024),
        nn.ReLU(),
        nn.BatchNorm1d(1024),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=1024, out_features=1)
    )
    
    return model

def get_configured_model_resnext(model_name, pretrained=True):
    model = select_model(model_name, pretrained)
    model.fc = nn.Linear(in_features=2048, out_features=1)
    return model

In [ ]:
# Get data
images = train_labels.img_path.values
targets = train_labels.target.values

aug = None
batch_size = 64

train_imgs, val_imgs, train_targets, val_targets = train_test_split(images, targets, 
                                                                    stratify=targets,
                                                                    random_state=42)

train_dataset = ET_Dataset(img_paths=train_imgs,
                                     targets=train_targets,
                                     resize=(224, 224),
                                     augmentations=aug)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=4)

val_dataset = ET_Dataset(img_paths=val_imgs,
                                     targets=val_targets,
                                     resize=(224, 224),
                                     augmentations=aug)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                          batch_size=batch_size,
                                          shuffle=False, num_workers=4)

# Configure model
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 10
loss_fn = nn.BCEWithLogitsLoss()
lr = 0.005

model = get_configured_model_resnext(model_name = "resnext50", pretrained=True)
# Configure input layer as input has 6 channels and not 3
model.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr, amsgrad=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)

In [ ]:
epoch_loss = []
for epoch in range(epochs):
    mean_loss_batch = train_model(train_loader, model, optimizer, loss_fn, device)
    scheduler.step(mean_loss_batch)
    epoch_loss.append(mean_loss_batch)
    print("Epoch : {0}, Current Mean Loss : {1}".format(epoch+1, mean_loss_batch))
roc_val = evaluate_model(train_loader, model, device)
print("ROC AUC value is : ", roc_val)

In [ ]:
def plot_training_details(epoch_loss, lr, batch_size, epochs):
    
    plt.plot(epoch_loss)
    plt.xlabel('Loss')
    plt.ylabel('Epochs')
    plt.title('Loss vs Epoch')
    
    print("Batch size {0}, LR = {1} and Epochs = {2}".format(batch_size, lr, epochs))

In [ ]:
plot_training_details(epoch_loss, lr, batch_size, epochs)

In [ ]:
## -- RESNET 18 --
# Epoch = 10, LR = 1e-3, Batch size = 64, Loss = 0.52, ROC AUC = 0.57

## -- RESMET 50 --
# Epoch = 10, LR = 1e-2, Batch size = 64, Loss = 0.177, ROC AUC = 0.6025
# Epoch = 10, LR = 0.005, Batch size = 64, Loss = , ROC AUC = 

In [ ]:
def evaluate_model_test(data_loader, model, device):
    
    # Set the model to evaluation mode
    model.eval()
    
    _actual = []
    _preds = []
    
    with torch.no_grad():
        for data in tqdm(data_loader, leave=True, desc='Evaluating'):
            inputs = data["image"]
            target = data["target"]
            
            inputs = inputs.to(device, dtype=torch.float)
            target = target.to(device, dtype=torch.float)
            
            pred = model(inputs)
            
            target = target.detach().cpu().numpy().tolist()
            pred = pred.detach().cpu().numpy().tolist()
            
            _actual.extend(target)
            _preds.extend(pred)
    
    return _preds, _actual

In [ ]:
submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path'] = submission['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')

test_images = submission.img_path.values

dummy_targets = submission.target.values

test_dataset = ET_Dataset(img_paths=test_images,
                                     targets=dummy_targets,
                                     resize=(224, 224),
                                     augmentations=aug)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=4)


In [ ]:
predictions, valid_targets = evaluate_model_test(test_loader, model, device=device)

predictions = np.array(predictions)

predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())

In [ ]:
submission.target = predictions

submission.drop(['img_path'], axis=1, inplace=True)

submission.to_csv('submission.csv', index=False)

In [ ]:
torch.save(model.state_dict(), 'model_0.63_state_dict')

In [ ]:
torch.save(model, 'entire_model_0.63')